In [34]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [35]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Deborshi
[nltk_data]    |     Roy\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\Deborshi
[nltk_data]    |     Roy\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Deborshi
[nltk_data]    |     Roy\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Deborshi
[nltk_data]    |     Roy\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\Deborshi
[nltk_data]    |     Roy\AppD

True

# Initialise the constants

In [30]:
SRC_TRAIN_CSV_FILE_PATH = '../data/source/twitter_training.csv'
SRC_TEST_CSV_FILE_PATH = '../data/source/twitter_validation.csv'

HEADERS = ['Tweet ID', 'entity', 'sentiment', 'Tweet content']
SENTIMENT_ENCODE = {
    'Positive': 0,
    'Neutral': 1,
    'Negative': 2,
    'Irrelevant': 3
}

# Reading the file

In [21]:
df = pd.read_csv(SRC_TRAIN_CSV_FILE_PATH, names=HEADERS)
df.shape

(74682, 4)

In [22]:
df.head()

,Tweet ID,entity,sentiment,Tweet content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


## Getting the structure of data and further analysing

1. df.info() -> to get datatypes of each column.
2. Getting the unique values of ```sentiment``` columns, and create the mapping variable descibe in the next step.
3. changing the sentiment to integer values, please refer ```SENTIMENT_ENCODE``` variable.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet ID       74682 non-null  int64 
 1   entity         74682 non-null  object
 2   sentiment      74682 non-null  object
 3   Tweet content  73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [29]:
## Getting the unique values for sentiment
for i in list(df['sentiment'].unique()):
    print(i)    

Positive
Neutral
Negative
Irrelevant


In [31]:
## converting the columns to integers
df['sentiment'] = df['sentiment'].map(SENTIMENT_ENCODE)

In [33]:
df.describe()

,Tweet ID,sentiment
count,74682.000000,74682.000000
mean,6432.586165,1.370772
std,3740.427870,1.067275
min,1.000000,0.000000
25%,3195.000000,0.000000
50%,6422.000000,1.000000
75%,9601.000000,2.000000
max,13200.000000,3.000000
